# Create dataset of expressions for training similar-expressions model

## Using custom generator (based on NeSymRes and DLSM code)

Creates .txt file with string expressions.

In [226]:
from importlib import reload
from custom import utils, generator, dclasses
reload(generator)
reload(utils)
reload(dclasses)
from custom.utils import create_generator
import os
import numpy as np
np.random.seed(2)

gen, params = create_generator('./custom/data_settings.json')
# gen.generate_equation(np.random, -1e3, 1e3, simplify=False, check_const_range=True)

In [ ]:
# Generate data
expressions = [gen.generate_equation(np.random, -1, 1, simplify=False, check_const_range=True) for _ in range(100000)]

In [227]:
datapath = '/Users/luis/Desktop/Cranmer 2024/Workplace/smallMutations/similar-expressions/data'

# Filter out empty strings
expressions = [expr for expr in expressions if expr]

# Save
output_file = os.path.join(datapath, 'expr_240807_5.txt')
with open(output_file, 'w') as f:
    for expr in expressions:
        f.write(f"{expr}\n")

## Parse string to onehot representation

### Whole dataset

In [2]:
import pandas as pd
import h5py
from parsing import parse_dataset

In [4]:
# Load data
datapath = r'/Users/luis/Desktop/Cranmer 2024/Workplace/smallMutations/similar-expressions/data/expr_240807_5.txt'
with open(datapath, 'r') as f:
    eqs = f.readlines()
    eqs = [eq.strip('\n') for eq in eqs]

# Parse dataset
onehot, invalid, consts = parse_dataset(eqs)

# Save parsed dataset
with h5py.File(f'{datapath[:-4]}-parsed.h5', 'w') as f:
    f.create_dataset('data', data=onehot)
    f.create_dataset('invalid_indices', data=invalid)

In [ ]:
a = [float(const) for const_list in consts for const in const_list]
s = pd.Series(a)
s.describe()

In [ ]:
from importlib import reload
import util
reload(util)
from util import plot_onehot

plot_onehot(onehot[3, ...], GCFG)